In [23]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import r2_score
from sklearn import svm
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
#from sklearn.naive_bayes import GaussianNBf
from sklearn import metrics 
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import stats
#from sklearn.metrics import jaccard_similarity_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss

In [24]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score
import lightgbm as lgb
import xgboost as xgb

In [25]:
!pip install lightgbm


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [26]:
df=pd.read_csv(r"C:\Users\beher\jupyterZ\wellsfargo_loan\lei_KB1H1DSPRFMYMCUFXT09.csv")

C:\Users\beher\AppData\Local\Temp\ipykernel_11892\2570174479.py:1: DtypeWarning: Columns (43,46,77,78,79,80) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(r"C:\Users\beher\jupyterZ\wellsfargo_loan\lei_KB1H1DSPRFMYMCUFXT09.csv")


In [27]:
df

,activity_year,lei,derived_msa-md,state_code,county_code,census_tract,conforming_loan_limit,derived_loan_product_type,derived_dwelling_category,derived_ethnicity,...,denial_reason-2,denial_reason-3,denial_reason-4,tract_population,tract_minority_population_percent,ffiec_msa_md_median_family_income,tract_to_msa_income_percentage,tract_owner_occupied_units,tract_one_to_four_family_homes,tract_median_age_of_housing_units
0,2018,KB1H1DSPRFMYMCUFXT09,99999,CO,8055.0,8.055961e+09,C,FHA:First Lien,Single Family (1-4 Units):Manufactured,Joint,...,NaN,NaN,NaN,3475,11.40,67300,104,1510,3247,29
1,2018,KB1H1DSPRFMYMCUFXT09,99999,AL,1049.0,1.049960e+09,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,6915,8.86,48500,87,1914,2759,32
2,2018,KB1H1DSPRFMYMCUFXT09,99999,MS,28145.0,2.814595e+10,C,VA:First Lien,Single Family (1-4 Units):Site-Built,Hispanic or Latino,...,NaN,NaN,NaN,5470,9.40,47000,75,1619,2328,32
3,2018,KB1H1DSPRFMYMCUFXT09,99999,MO,29147.0,2.914747e+10,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,2288,2.75,53100,107,694,1187,72
4,2018,KB1H1DSPRFMYMCUFXT09,99999,SC,45009.0,4.500996e+10,C,FHA:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,7137,60.94,48300,94,2088,3125,36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
966466,2018,KB1H1DSPRFMYMCUFXT09,31084,CA,6037.0,6.037190e+09,NaN,Conventional:First Lien,Multifamily:Site-Built,Ethnicity Not Available,...,NaN,NaN,NaN,4006,24.16,69300,97,317,584,58
966467,2018,KB1H1DSPRFMYMCUFXT09,42644,WA,53033.0,5.303300e+10,NaN,Conventional:First Lien,Multifamily:Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,8233,15.13,103400,104,2033,2679,60
966468,2018,KB1H1DSPRFMYMCUFXT09,35614,NY,36005.0,3.600503e+10,NaN,Conventional:First Lien,Multifamily:Site-Built,Ethnicity Not Available,...,NaN,NaN,NaN,3233,52.58,78000,84,255,279,57
966469,2018,KB1H1DSPRFMYMCUFXT09,33124,FL,12086.0,1.208600e+10,NaN,Conventional:First Lien,Multifamily:Site-Built,Ethnicity Not Available,...,NaN,NaN,NaN,7383,99.01,52300,74,832,1357,47


### make the binary variable

In [28]:
column = []
for num in df["action_taken"]:
    if num == 2 or num == 6:
        column.append(1)
    elif num == 3 or num == 4:
        column.append(0)
    else:
        column.append(2)
        
#rid of 1, 4, 7, 8
#approved = 2, 6
#denied = 3, 4

In [29]:
df["action"] = column

### drop other array

In [30]:
df.drop(df[df['action'] == 2].index, inplace = True) 

### making equal shares 

In [31]:
df['action'].value_counts()

action
1    439793
0    247040
Name: count, dtype: int64

In [32]:
df.describe()

,activity_year,derived_msa-md,county_code,census_tract,action_taken,purchaser_type,preapproval,loan_type,loan_purpose,lien_status,...,denial_reason-3,denial_reason-4,tract_population,tract_minority_population_percent,ffiec_msa_md_median_family_income,tract_to_msa_income_percentage,tract_owner_occupied_units,tract_one_to_four_family_homes,tract_median_age_of_housing_units,action
count,686833.0,686833.000000,686777.000000,6.867770e+05,686833.000000,686833.000000,686833.0,686833.000000,686833.00000,686833.000000,...,20834.000000,2270.000000,686833.000000,686833.000000,686833.000000,686833.000000,686833.000000,686833.000000,686833.000000,686833.000000
mean,2018.0,35734.666522,27114.757731,2.711495e+10,5.023285,0.899331,2.0,1.256623,8.42181,1.136937,...,3.867812,3.959471,5923.348463,34.164126,75833.506544,116.648407,1506.994427,2037.426274,34.326151,0.640320
std,0.0,21331.424826,16844.282078,1.684430e+10,1.324357,1.070691,0.0,0.613104,12.41799,0.343781,...,3.014787,2.834985,3459.900159,25.927443,15687.753180,42.610771,951.989433,1170.684414,18.013253,0.479907
min,2018.0,0.000000,1001.000000,1.001020e+09,2.000000,0.000000,2.0,1.000000,1.00000,1.000000,...,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,0.000000
25%,2018.0,19804.000000,12031.000000,1.203102e+10,4.000000,0.000000,2.0,1.000000,1.00000,1.000000,...,1.000000,1.000000,3925.000000,13.320000,65100.000000,89.000000,931.000000,1335.000000,20.000000,0.000000
50%,2018.0,33874.000000,27053.000000,2.705303e+10,6.000000,1.000000,2.0,1.000000,1.00000,1.000000,...,4.000000,3.000000,5272.000000,26.790000,74400.000000,111.000000,1345.000000,1825.000000,32.000000,1.000000
75%,2018.0,41540.000000,42101.000000,4.210100e+10,6.000000,1.000000,2.0,1.000000,5.00000,1.000000,...,6.000000,6.000000,6961.000000,49.520000,81800.000000,138.000000,1854.000000,2461.000000,46.000000,1.000000
max,2018.0,99999.000000,72153.000000,7.215375e+10,6.000000,9.000000,2.0,4.000000,32.00000,2.000000,...,9.000000,9.000000,53812.000000,100.000000,134800.000000,507.000000,19529.000000,25391.000000,76.000000,1.000000


In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 686833 entries, 0 to 966470
Data columns (total 100 columns):
 #   Column                                    Non-Null Count   Dtype  
---  ------                                    --------------   -----  
 0   activity_year                             686833 non-null  int64  
 1   lei                                       686833 non-null  object 
 2   derived_msa-md                            686833 non-null  int64  
 3   state_code                                686765 non-null  object 
 4   county_code                               686777 non-null  float64
 5   census_tract                              686777 non-null  float64
 6   conforming_loan_limit                     686044 non-null  object 
 7   derived_loan_product_type                 686833 non-null  object 
 8   derived_dwelling_category                 686833 non-null  object 
 9   derived_ethnicity                         686833 non-null  object 
 10  derived_race            

In [34]:
numeric_df = df.select_dtypes(include=['number'])

# Compute the correlation matrix for numerical columns only
correlation_matrix = numeric_df.corr()

# Print the correlation matrix
print(correlation_matrix)

                                   activity_year  derived_msa-md  county_code  \
activity_year                                NaN             NaN          NaN   
derived_msa-md                               NaN        1.000000     0.033564   
county_code                                  NaN        0.033564     1.000000   
census_tract                                 NaN        0.033570     1.000000   
action_taken                                 NaN       -0.026452     0.090479   
...                                          ...             ...          ...   
tract_to_msa_income_percentage               NaN       -0.055532    -0.002409   
tract_owner_occupied_units                   NaN       -0.099280     0.072725   
tract_one_to_four_family_homes               NaN       -0.008832     0.037048   
tract_median_age_of_housing_units            NaN        0.073277    -0.008698   
action                                       NaN       -0.026404     0.090985   

                           

In [35]:
df.columns

Index(['activity_year', 'lei', 'derived_msa-md', 'state_code', 'county_code',
       'census_tract', 'conforming_loan_limit', 'derived_loan_product_type',
       'derived_dwelling_category', 'derived_ethnicity', 'derived_race',
       'derived_sex', 'action_taken', 'purchaser_type', 'preapproval',
       'loan_type', 'loan_purpose', 'lien_status', 'reverse_mortgage',
       'open-end_line_of_credit', 'business_or_commercial_purpose',
       'loan_amount', 'loan_to_value_ratio', 'interest_rate', 'rate_spread',
       'hoepa_status', 'total_loan_costs', 'total_points_and_fees',
       'origination_charges', 'discount_points', 'lender_credits', 'loan_term',
       'prepayment_penalty_term', 'intro_rate_period', 'negative_amortization',
       'interest_only_payment', 'balloon_payment',
       'other_nonamortizing_features', 'property_value', 'construction_method',
       'occupancy_type', 'manufactured_home_secured_property_type',
       'manufactured_home_land_property_interest', 'total_

In [36]:
df_new=df[['income','loan_amount','property_value','loan_type','tract_owner_occupied_units','loan_purpose','business_or_commercial_purpose']]

In [37]:
df_new

,income,loan_amount,property_value,loan_type,tract_owner_occupied_units,loan_purpose,business_or_commercial_purpose
0,NaN,65000.0,75000.0,2,1510,1,2
1,79.0,75000.0,NaN,1,1914,1,2
3,88.0,25000.0,25000.0,1,694,1,2
4,48.0,135000.0,145000.0,2,2088,1,2
5,NaN,135000.0,165000.0,2,1046,5,2
...,...,...,...,...,...,...,...
966461,NaN,29175000.0,NaN,1,2018,1,1
966462,NaN,6205000.0,NaN,1,1081,31,1
966466,NaN,6255000.0,NaN,1,317,1,1
966467,NaN,3005000.0,NaN,1,2033,1,1


In [38]:
df_new.isna().sum()

income                            68262
loan_amount                           0
property_value                    86317
loan_type                             0
tract_owner_occupied_units            0
loan_purpose                          0
business_or_commercial_purpose        0
dtype: int64

In [39]:
median_salary = df_new['income'].median()

# Fill NaN values in 'salary' column with the calculated median
df_new['income'].fillna(median_salary, inplace=True)

C:\Users\beher\AppData\Local\Temp\ipykernel_11892\3717044609.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['income'].fillna(median_salary, inplace=True)


In [40]:
df_new.isna().sum()

income                                0
loan_amount                           0
property_value                    86317
loan_type                             0
tract_owner_occupied_units            0
loan_purpose                          0
business_or_commercial_purpose        0
dtype: int64

In [41]:
median_salary = df_new['property_value'].median()

# Fill NaN values in 'salary' column with the calculated median
df_new['property_value'].fillna(median_salary, inplace=True)

C:\Users\beher\AppData\Local\Temp\ipykernel_11892\1914270780.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['property_value'].fillna(median_salary, inplace=True)


In [42]:
df_new.isna().sum()

income                            0
loan_amount                       0
property_value                    0
loan_type                         0
tract_owner_occupied_units        0
loan_purpose                      0
business_or_commercial_purpose    0
dtype: int64

In [43]:
X=np.asarray(df_new)
Y=np.asarray(df['action'])

In [44]:
print("X.shape:",X.shape)
print("Y.shape:",Y.shape)

X.shape: (686833, 7)
Y.shape: (686833,)


In [45]:
X = preprocessing.StandardScaler().fit(X).transform(X)

In [46]:
X

array([[-2.83987878e-02, -1.48355457e-01, -5.29456275e-01, ...,
         3.15715226e-03, -5.97666390e-01,  3.70987330e-02],
       [-3.75703409e-02, -1.39839894e-01, -1.40801256e-01, ...,
         4.27531919e-01, -5.97666390e-01,  3.70987330e-02],
       [-2.83987878e-02, -1.82417706e-01, -6.26620030e-01, ...,
        -8.53995843e-01, -5.97666390e-01,  3.70987330e-02],
       ...,
       [-2.83987878e-02,  5.12277769e+00, -1.40801256e-01, ...,
        -1.25000893e+00, -5.97666390e-01, -2.69550984e+01],
       [-2.83987878e-02,  2.35521990e+00, -1.40801256e-01, ...,
         5.52533398e-01, -5.97666390e-01, -2.69550984e+01],
       [-2.83987878e-02,  4.82473300e+00, -1.40801256e-01, ...,
        -9.10719203e-01, -5.97666390e-01, -2.69550984e+01]])

In [47]:
# Here we use 30% of the data for testing and 70% of the data for testing 
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=42)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

Train set: (549466, 7) (549466,)
Test set: (137367, 7) (137367,)


In [48]:
from sklearn.metrics import accuracy_score, f1_score

In [49]:
rf_clf = RandomForestClassifier(random_state=42)
rf_clf.fit(X_train, y_train)
rf_predictions = rf_clf.predict(X_test)

In [50]:
rf_accuracy = accuracy_score(y_test, rf_predictions)
rf_f1 = f1_score(y_test, rf_predictions)

In [51]:
print("Random Forest Classifier:")
print(f"Accuracy: {rf_accuracy}")
print(f"F1 Score: {rf_f1}")
print("--------------------------")

Random Forest Classifier:
Accuracy: 0.9065932865972177
F1 Score: 0.9283804526806397
--------------------------


In [52]:
# LightGBM Classifier
lgb_clf = lgb.LGBMClassifier(random_state=42)
lgb_clf.fit(X_train, y_train)
lgb_predictions = lgb_clf.predict(X_test)

[LightGBM] [Info] Number of positive: 351834, number of negative: 197632
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006198 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1035
[LightGBM] [Info] Number of data points in the train set: 549466, number of used features: 7
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.640320 -> initscore=0.576753
[LightGBM] [Info] Start training from score 0.576753


In [53]:
lgb_accuracy = accuracy_score(y_test, lgb_predictions)
lgb_f1 = f1_score(y_test, lgb_predictions)

In [54]:
print("LightGBM Classifier:")
print(f"Accuracy: {lgb_accuracy}")
print(f"F1 Score: {lgb_f1}")
print("--------------------------")

LightGBM Classifier:
Accuracy: 0.9095852715717749
F1 Score: 0.9312528367891421
--------------------------


In [55]:
# XGBoost Classifier
xgb_clf = xgb.XGBClassifier(random_state=42)
xgb_clf.fit(X_train, y_train)
xgb_predictions = xgb_clf.predict(X_test)

In [56]:
xgb_accuracy = accuracy_score(y_test, xgb_predictions)
xgb_f1 = f1_score(y_test, xgb_predictions)

In [57]:
print("XGBoost Classifier:")
print(f"Accuracy: {xgb_accuracy}")
print(f"F1 Score: {xgb_f1}")
print("--------------------------")

XGBoost Classifier:
Accuracy: 0.9126354946966885
F1 Score: 0.933447203074483
--------------------------


In [58]:
# K-Nearest Neighbors (KNN) Classifier
knn_clf = KNeighborsClassifier()
knn_clf.fit(X_train, y_train)
knn_predictions = knn_clf.predict(X_test)

In [59]:
knn_accuracy = accuracy_score(y_test, knn_predictions)
knn_f1 = f1_score(y_test, knn_predictions)

In [60]:
print("K-Nearest Neighbors (KNN) Classifier:")
print(f"Accuracy: {knn_accuracy}")
print(f"F1 Score: {knn_f1}")
print("--------------------------")

K-Nearest Neighbors (KNN) Classifier:
Accuracy: 0.8873601374420348
F1 Score: 0.9140527359480972
--------------------------


In [25]:
!pip install xgboost


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip
